<a href="https://colab.research.google.com/github/markus-hoanglong/test-gitpage/blob/master/instant_ngp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/ceres-solver-v2.zip
!unzip /content/ceres-solver-v2.zip -d ceres-solver

!cp -r /content/ceres-solver/lib/. /usr/local/lib
!chmod 755 /content/ceres-solver/bin/colmap
!cp -r /content/ceres-solver/bin/. /usr/local/bin

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v2.zip
!unzip /content/instant-ngp-v2.zip -d instant-ngp

!pip install commentjson huggingface-hub
%cd /content/instant-ngp


In [10]:
!pip install commentjson imageio numpy opencv-python-headless pybind11 pyquaternion scipy tqdm pyngp

ERROR: Could not find a version that satisfies the requirement pyngp (from versions: none)
ERROR: No matching distribution found for pyngp


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [9]:
import os

scene_path = "/content/gdrive/MyDrive/fox" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(train_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {train_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/instant-ngp/./scripts/colmap2nerf.py:71: SyntaxWarning: invalid escape sequence '\,'
  time_slice_value = f",select='between(t\,{start}\,{end})'"
/content/instant-ngp/./scripts/colmap2nerf.py:71: SyntaxWarning: invalid escape sequence '\,'
  time_slice_value = f",select='between(t\,{start}\,{end})'"
running colmap with:
	db=colmap.db
	images="/content/train"
	sparse=colmap_sparse
	text=colmap_text
warning! folders 'colmap_sparse' and 'colmap_text' will be deleted/replaced. continue? (Y/n)Y
==== running: colmap feature_extractor --ImageReader.camera_model OPENCV --ImageReader.camera_params "" --SiftExtraction.estimate_affine_shape=true --SiftExtraction.domain_size_pooling=true --ImageReader.single_camera 1 --database_path colmap.db --image_path "/content/train"
colmap: error while loading shared libraries: libGLEW.so.2.1: cannot open shared object f